In [1]:
import pandas as pd
dataset_train=pd.read_csv('/kaggle/input/yelp-csv/train.csv')

In [2]:
# I need rewies in a list (no labels required in this part)
data_text=dataset_train['text']
data_text['index']=data_text.index
data_text.head()


/tmp/ipykernel_18/3086331156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index']=data_text.index


0    dr. goldberg offers everything i look for in a...
1    Unfortunately, the frustration of being Dr. Go...
2    Been going to Dr. Goldberg for over 10 years. ...
3    Got a letter in the mail last week that said D...
4    I don't know what Dr. Goldberg was like before...
Name: text, dtype: object

In [3]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [4]:
#Generate corpus for lda, preprocessed and lemmatized
#Corpus is the collection of documents used in training, Porterstemmer performs lemmatization
corpus=[]
for i in range(0,len(data_text)-1):
    text=re.sub('[^a-zA-z]',' ',data_text[i])
    text=text.lower()
    text=text.split()
    text=[PorterStemmer().stem(word) for word in text if not word in stopwords.words('english')]
    text=' '.join(text)
    corpus.append(text)

In [5]:
#Save corpus, since it is the part which needs more time for being executed. Next time we can directly load the data
import json

with open("corpus.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(corpus, f, indent=2) 

with open("corpus.json", 'r') as f:
    corpus = json.load(f) 

In [6]:
# I need to perform this operation because gensim.Dictionary takes as input array of tokens
processed_docs=[]
for i in range(0,len(data_text)-1):
    text = corpus[i].split()
    processed_docs.append(text)
    

In [7]:
import gensim
dictionary=gensim.corpora.Dictionary(processed_docs)


In [8]:
count=0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>20:
        break

0 affili
1 alway
2 blank
3 case
4 complaint
5 dr
6 draw
7 easi
8 everyth
9 explain
10 first
11 gener
12 get
13 goldberg
14 happen
15 hospit
16 import
17 look
18 need
19 nice
20 notch


In [9]:
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=1000)

In [10]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]

In [11]:
# Prepare corpus for a tfidf lda (term frequency–inverse document frequency)- More attention
#to terms which appear in the document but are less frequent.
from gensim import corpora,models
tfidf=models.TfidfModel(bow_corpus)

In [12]:
corpus_tfidf=tfidf[bow_corpus]

In [13]:
lda_model=gensim.models.LdaMulticore(bow_corpus,
                                    num_topics=20,
                                    id2word=dictionary,
                                    passes=2,
                                    random_state=11
                                    )
lda_model.save('lda_model.model')

In [14]:
for idx,topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.059*"coffe" + 0.047*"breakfast" + 0.023*"tea" + 0.018*"lunch" + 0.016*"cafe" + 0.016*"cup" + 0.016*"morn" + 0.016*"juic" + 0.015*"fresh" + 0.013*"open"


Topic: 1 
Words: 0.020*"wine" + 0.019*"delici" + 0.017*"dinner" + 0.015*"dish" + 0.015*"meal" + 0.015*"amaz" + 0.011*"excel" + 0.011*"dessert" + 0.011*"enjoy" + 0.011*"appet"


Topic: 2 
Words: 0.027*"show" + 0.017*"club" + 0.013*"friend" + 0.012*"guy" + 0.012*"fun" + 0.012*"line" + 0.012*"girl" + 0.011*"music" + 0.010*"cool" + 0.010*"danc"


Topic: 3 
Words: 0.034*"chees" + 0.034*"salad" + 0.033*"sauc" + 0.028*"meat" + 0.019*"chicken" + 0.017*"side" + 0.016*"flavor" + 0.014*"bread" + 0.014*"bbq" + 0.013*"potato"


Topic: 4 
Words: 0.098*"room" + 0.050*"hotel" + 0.049*"stay" + 0.020*"pool" + 0.017*"check" + 0.017*"clean" + 0.015*"floor" + 0.014*"bed" + 0.013*"bathroom" + 0.012*"strip"


Topic: 5 
Words: 0.355*"e" + 0.107*"de" + 0.092*"la" + 0.047*"f" + 0.047*"c" + 0.043*"die" + 0.038*"pour" + 0.037*"n" + 0.020*"b" +

In [15]:
# lda with tfidf
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=20, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")
    
    

Topic: 0 Word: 0.035*"casino" + 0.025*"scottsdal" + 0.019*"locat" + 0.013*"downtown" + 0.013*"close" + 0.009*"town" + 0.009*"street" + 0.008*"strip" + 0.008*"open" + 0.007*"cheap"


Topic: 1 Word: 0.035*"breakfast" + 0.022*"egg" + 0.017*"pancak" + 0.014*"brunch" + 0.012*"bacon" + 0.012*"toast" + 0.012*"coffe" + 0.009*"juic" + 0.008*"french" + 0.008*"morn"


Topic: 2 Word: 0.020*"call" + 0.011*"phone" + 0.010*"told" + 0.009*"compani" + 0.008*"custom" + 0.007*"card" + 0.007*"appoint" + 0.007*"manag" + 0.007*"offic" + 0.007*"busi"


Topic: 3 Word: 0.073*"pizza" + 0.020*"wing" + 0.015*"crust" + 0.014*"dog" + 0.010*"slice" + 0.010*"chees" + 0.009*"pie" + 0.009*"sauc" + 0.008*"hot" + 0.007*"top"


Topic: 4 Word: 0.057*"e" + 0.040*"buffet" + 0.021*"de" + 0.015*"die" + 0.012*"crab" + 0.011*"f" + 0.010*"la" + 0.009*"seafood" + 0.009*"averag" + 0.008*"dessert"


Topic: 5 Word: 0.013*"steak" + 0.013*"bbq" + 0.012*"rib" + 0.011*"chees" + 0.010*"mac" + 0.010*"fri" + 0.010*"pork" + 0.009*"potato" + 

In [16]:
print(dataset_train['text'][11])
for index, score in sorted(lda_model[bow_corpus[11]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

This place is absolute garbage...  Half of the tees are not available, including all the grass tees.  It is cash only, and they sell the last bucket at 8, despite having lights.  And if you finish even a minute after 8, don't plan on getting a drink.  The vending machines are sold out (of course) and they sell drinks inside, but close the drawers at 8 on the dot.  There are weeds grown all over the place.  I noticed some sort of batting cage, but it looks like those are out of order as well.  Someone should buy this place and turn it into what it should be.

Score: 0.42214149236679077	 
Topic: 0.038*"custom" + 0.018*"employe" + 0.018*"walk" + 0.015*"help" + 0.014*"line" + 0.013*"manag" + 0.013*"counter" + 0.012*"busi" + 0.012*"card" + 0.011*"rude"

Score: 0.27001717686653137	 
Topic: 0.105*"pizza" + 0.051*"store" + 0.037*"shop" + 0.018*"buy" + 0.018*"item" + 0.014*"find" + 0.013*"crust" + 0.012*"product" + 0.010*"select" + 0.010*"new"

Score: 0.21490934491157532	 
Topic: 0.020*"wine" +